In [1]:
# Obseity Classification problem
# import necessary classification problem
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df =  pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
df.head()

In [3]:
# preprocessing

# cheking for Nulls
df.isna().sum()

df.duplicated().sum() #  there are some dulplicates

df = df.drop_duplicates() # removing duplicates

In [ ]:
# encoding and making dummy for the categorical variables
df.info()

In [5]:
# categorical columns that needs to be converted either with LE or OHE
le_columns = ['CAEC','SCC','CALC','NObeyesdad']
ohe_columns = ['Gender','family_history_with_overweight','FAVC','SMOKE','MTRANS']

In [6]:
# Functions for One Hot Encoding and Label Encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

def label_ecoder(data,my_list):
    encoder = LabelEncoder()

    for item in my_list:
        data[item] = encoder.fit_transform(data[item])

        print(f"column : {item}")
        print(f"Unique values before Encoding :{df[item].astype('category').cat.categories.tolist()}")
        print(f"Unique values after Encoding : {df[item].nunique()}")
    return data


def ohe_encoder(data, my_list):
    ohe = OneHotEncoder(sparse_output=False,drop = 'first')
    data = data.copy()
    for item in my_list:
        if item in data.columns:
            transformed = ohe.fit_transform(data[[item]])
            ohe_col = ohe.get_feature_names_out([item])

            df_ohe = pd.DataFrame(transformed, columns = ohe_col, index=data.index)

            data = pd.concat([data, df_ohe],axis = 1)

            data.drop(columns = [item], inplace = True)

    return data

In [ ]:
df1 = label_ecoder(df,le_columns)

In [8]:
final_df = ohe_encoder(df1,ohe_columns)

In [ ]:
final_df.info(50)

In [10]:
# Let us start with the Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay 

X = final_df.drop(columns = 'NObeyesdad', axis = 1)
y = final_df['NObeyesdad']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
# Basic Standardization
scaler = StandardScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [ ]:
# Building a logistic Regression

log_regression = LogisticRegression(max_iter= 250)

log_regression.fit(scaled_X_train,y_train)
log_pred = log_regression.predict(scaled_X_test)


print(classification_report(y_test,log_pred))

In [ ]:
# printing the Confusion Matrix
ConfusionMatrixDisplay.from_estimator(log_regression, scaled_X_test,y_test)